# Graph Creation for AGM Delivery Network Optimization

## Purpose
This notebook focuses on creating a graph data structure from AGM's delivery and pickup location data based on the stations within BART (as shown below). The graph will serve as a foundation for optimizing delivery routes, analyzing network connectivity, and identifying potential growth opportunities for AGM.

## Approach
We utilize delivery locations, customer data, and geographic information to construct a directed graph. Key steps include loading data from various sources (e.g., CSV files outlining lines, stations and travel times), preprocessing this data to align with our graph construction requirements, and building the graph using Neo4j library. This process aims to enable efficient route analysis and optimization in subsequent analyses.

![Bart Map](../data/external/bart_map.png)

# Setup

In [ ]:
import neo4j
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [ ]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    """
    Runs a select query and returns rows in a pandas dataframe.
    Pandas will convert all numeric values from postgres to float.
    If it will fit in an integer, change it to integer.

    Parameters:
    - query: the query to run
    - rollback_before_flag: if True, rollback the transaction before running the query
    - rollback_after_flag: if True, rollback the transaction after running the query

    Returns:
    - a pandas dataframe with the results of the query
    """
        
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [ ]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [ ]:
cursor = connection.cursor()

In [ ]:
def my_read_csv_file(file_name, limit):
    """
    Reads a csv file and prints the first limit lines.
    """
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

# Data Loading and Pre-processing

## Overview
This section of the notebook involves the loading and initial setup of our primary datasets, which are crucial for constructing the graph data structure. The datasets include:

- **Stations**: Contains information about each station, including its name, geographical coordinates (latitude and longitude), and transfer time. This dataset forms the nodes of our graph, representing the physical locations where pickups and deliveries occur.
  
- **Lines**: Details the sequence of stations for each line, indicating the order in which stations are connected. This dataset helps in defining the edges of our graph, establishing the connections between nodes (stations).
  
- **Travel Times**: Provides the travel time between pairs of stations, essential for weighting the edges of our graph. This data allows us to model and optimize delivery routes based on actual travel times.

These datasets are stored in CSV format and are loaded into PostgreSQL tables for easy manipulation and querying throughout the graph creation process.

## Pre-processing Steps
The preprocessing steps involve preparing our database environment and ensuring the datasets are correctly structured for our analysis needs:

1. **Clearing Existing Tables**: We begin by dropping any existing tables related to stations, lines, and travel times. This ensures a clean slate for our analysis and prevents any data duplication issues.

2. **Creating Tables**: New tables are created for each of the three datasets, with appropriate data types and primary keys to ensure data integrity and facilitate efficient querying.

3. **Data Import**: The datasets are imported from their respective CSV files into the newly created tables. This step involves specifying the delimiter, handling null values, and ensuring the data aligns with the table structure.

In [ ]:
connection.rollback()

query = """

DROP TABLE IF EXISTS stations;
DROP TABLE IF EXISTS lines;
DROP TABLE IF EXISTS travel_times;

"""

cursor.execute(query)
connection.commit()

In [ ]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

copy stations
from '../data/raw/stations.csv' delimiter ',' NULL '' csv header;

copy lines
from '../data/raw/lines.csv' delimiter ',' NULL '' csv header;

copy travel_times
from '../data/raw/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)
connection.commit()

# Graph Creation

## Overview

This section is dedicated to constructing the graph data structure that will underpin our analysis of the delivery network. We utilize Neo4j, a graph database, to model the network as a graph of stations connected by lines, reflecting the real-world routes and transfer possibilities within the network. The process involves several key steps: clearing the existing graph data, creating nodes for stations and lines, and establishing relationships that represent the connections between stations, including their travel times and transfer times.

## Initial Setup

We start by establishing a connection to the Neo4j database, which serves as the backend for our graph. This involves specifying the URI and authentication details to access the database.

## Wiping the Database

To ensure a clean start, we first delete all existing nodes and relationships in the database. This is critical for preventing data duplication and ensuring that our analysis is based on the latest data.

## Creating Nodes

Nodes in our graph represent stations and lines. Each station has two nodes: a departure node and an arrival node, to accurately model the flow of travel through the station. Additionally, each line at a station is represented as a node, allowing us to model the journey on specific lines and the transfers between them.

### Stations and Lines

- **Station Nodes**: For each station, we create two nodes representing the points of departure and arrival. This differentiation allows us to model the wait times and transfer times accurately.
- **Line Nodes**: We also create nodes for each line at a station, enabling us to represent travels and transfers within the network.

## Creating Relationships

Relationships in our graph represent the connections between stations and lines, including the travel times and transfer times, modeled as weighted edges in the graph.

### One-way Relationships

For connections that represent a single direction of travel (e.g., from a departure node to a line node), we create one-way relationships with a weight indicating the travel or transfer time.

### Two-way Relationships

For direct connections between stations on the same line, we create two-way relationships to accurately model the bidirectional nature of travel. These relationships are also weighted by travel times.

## Populating the Graph

We populate the graph by iterating over our datasets:
- **Station and Line Nodes**: For each station listed in our dataset, we create the corresponding departure and arrival nodes, as well as nodes for each line passing through the station.
- **Transfer Relationships**: Based on the transfer times between lines at the same station, we establish relationships that model the transfer process, including the required time.
- **Segment Relationships**: For each pair of stations connected by a line, we create relationships representing the segments of travel, including the travel time.

This comprehensive approach to graph creation enables us to model the delivery network with high fidelity, laying the groundwork for subsequent analyses focused on optimizing delivery routes and understanding network dynamics.


In [ ]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [ ]:
session = driver.session(database="neo4j")

In [ ]:
def my_neo4j_wipe_out_database():
    """Wipe out database by deleting all nodes and relationships"""
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [ ]:
def my_neo4j_run_query_pandas(query, **kwargs):
    """Run a query and return the results in a pandas dataframe"""
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [ ]:
def my_neo4j_create_node(station_name):
    """Create a node with label Station"""
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [ ]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    """Create a relationship one way between two stations with a weight"""
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [ ]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    """Create relationships two way between two stations with a weight"""
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [ ]:
my_neo4j_wipe_out_database()

In [ ]:
# create departure and arrival nodes
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [ ]:
# create line nodes and relationships with departure and arrival
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [ ]:
# create transfer lines
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [ ]:
# create segments
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)